In [1]:
#hide
#default_exp config.hpconfig
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=[])

# Loading experiment manager and configuration

Load last experiment manager, experiment data, and configuration

In [2]:
#export 
import os
import hpsearch
from hpsearch.config.manager_factory import ManagerFactory
from hpsearch.config import manager_factory as mf
import hpsearch.config.hp_defaults as dflt

changeable_properties = {'folder', 'metric', 'manager_path', 'op', 'path_experiments', 'parent_path', 'backend'}

In [3]:
#for tests
from sklearn.utils import Bunch

from dsblocks.utils.utils import check_last_part, remove_previous_results

from hpsearch.examples.dummy_experiment_manager import DummyExperimentManager, generate_data

## get_experiment_manager

In [4]:
#export
def get_experiment_manager (manager_path=dflt.manager_path, **kwargs):
    manager_factory = ManagerFactory(manager_path=manager_path)
    experiment_manager = manager_factory.get_experiment_manager()
    if len(kwargs) > 0: modify_experiment_manager (experiment_manager, **kwargs)
    mf.experiment_manager = experiment_manager
    return mf.experiment_manager

### Usage example

In [5]:
#exports tests.config.test_hpconfig
def test_get_experiment_manager ():
    em = generate_data ('get_experiment_manager')
    path_experiments = str(em.path_experiments.parent)
    manager_path = em.manager_path
    em_orig = em
    del em
    
    em = get_experiment_manager (manager_path)
    check_last_part (em.path_experiments, 'test_get_experiment_manager/default')
    assert em.key_score == 'validation_accuracy' and em.op == 'max'

    em = get_experiment_manager (manager_path, path_experiments='my_new_parent_path/my_new_folder', 
                               metric='new_metric', op='min')
    check_last_part (em.path_experiments, 'my_new_parent_path/my_new_folder')
    assert em.key_score == 'new_metric' and em.op == 'min'

    em = get_experiment_manager (manager_path, folder='other_folder')
    check_last_part (em.path_experiments, 'my_new_parent_path/other_folder')
    assert em.key_score == 'new_metric' and em.op == 'min'
    remove_previous_results (path_experiments)

In [6]:
tst.run (test_get_experiment_manager, tag='dummy')

## get_default_parameters

In [7]:
#export
def get_default_parameters (parameters, manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.get_default_parameters (parameters)

## get_path_experiments and other paths

In [8]:
#export
def get_path_experiments (manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.path_experiments

def get_path_alternative (path_results, manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.get_path_alternative (path_results)

def get_path_experiment (experiment_id, manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.get_path_experiment (experiment_id)

def get_path_results (experiment_id, run_number, manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.get_path_results (experiment_id, run_number)

def get_path_data (run_number, parameters={}, manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.get_path_data (run_number, parameters)

## experiment_visualization

In [9]:
#export
def experiment_visualization (manager_path=dflt.manager_path, **kwargs):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.experiment_visualization (**kwargs)

## get_experiment_data

In [10]:
#export
def get_experiment_data (experiments=None, 
                         manager_path=dflt.manager_path):
    experiment_manager = get_experiment_manager (manager_path=manager_path)
    return experiment_manager.get_experiment_data (experiments=experiments)

## get_em_args

In [11]:
#export
def get_em_args (pars):
    if not isinstance (pars, dict): pars = vars(pars)
    return {k:pars[k] for k in set(pars).intersection(changeable_properties) if pars[k] is not None}

### Usage example

In [12]:
#exports tests.config.test_hpconfig
def test_get_em_args ():
    em_args = get_em_args ({'hello': 1, 'path_experiments': 'mypath', 'folder': None})
    assert (em_args=={'path_experiments': 'mypath'})

In [13]:
tst.run (test_get_em_args, tag='dummy')

## modify_experiment_manager

In [14]:
#export
def modify_experiment_manager (experiment_manager, **kwargs):
    path_experiments_keys = {'path_experiments', 'parent_path', 'folder'}
    path_experiments_args = {k:kwargs[k] for k in path_experiments_keys.intersection(kwargs)}
    if len(path_experiments_args) > 0: experiment_manager.set_path_experiments (**path_experiments_args)
    for k in set(kwargs) - path_experiments_keys:
        if k == 'metric': setattr (experiment_manager, 'key_score', kwargs[k])
        else: setattr (experiment_manager, k, kwargs[k])

### Usage example

In [15]:
#exports tests.config.test_hpconfig
def test_modify_experiment_manager ():
    em = DummyExperimentManager ()
    check_last_part (em.path_experiments, 'results')
    assert em.key_score == 'validation_accuracy' and em.op == 'max'
    
    modify_experiment_manager (em, path_experiments='my_new_parent_path/my_new_folder', 
                               metric='new_metric', op='min')
    check_last_part (em.path_experiments, 'my_new_parent_path/my_new_folder')
    assert em.key_score == 'new_metric' and em.op == 'min'
    
    modify_experiment_manager (em, folder='other_folder')
    check_last_part (em.path_experiments, 'my_new_parent_path/other_folder')
    assert em.key_score == 'new_metric' and em.op == 'min'

In [16]:
tst.run (test_modify_experiment_manager, tag='dummy')

## add_em_args

In [17]:
#export 
def add_em_args (parser, but=None):
    if isinstance (but, str): but = [but]
    properties = changeable_properties if but is None else changeable_properties - set(but)
    for k in properties:
        parser.add_argument(f'--{k}', type=str, default=None)

### Usage example

In [12]:
#exports tests.config.test_hpconfig
def test_add_em_args ():
    import argparse
    
    # by default, add multiple string parameters
    parser = argparse.ArgumentParser(description='test')
    add_em_args (parser)
    pars = parser.parse_args([])
    assert hasattr(pars, 'metric') and hasattr(pars, 'manager_path')

    # we can skip some of those using `but`:
    parser = argparse.ArgumentParser(description='test)
    add_em_args (parser, but=['metric'])
    pars = parser.parse_args([])
    assert not hasattr(pars, 'metric') and hasattr(pars, 'manager_path')

In [13]:
tst.run (test_add_em_args, tag='dummy')

## register_manager

In [18]:
#export
def register_manager (experiment_manager_to_register, manager_path=dflt.manager_path, **kwargs):
    manager_factory = ManagerFactory(manager_path=manager_path)
    manager_factory.register_manager (experiment_manager_to_register)

### Usage example

In [19]:
#exports tests.config.test_hpconfig
def test_register_manager ():
    em = generate_data ('register_manager')
    manager_path = em.manager_path
    em_orig = em
    del em
    em = get_experiment_manager (manager_path)
    assert em is not None
    
    register_manager (None)
    import hpsearch.config.hpconfig as hpcfg
    assert hpcfg.mf.experiment_manager is None
    
    em_orig.remove_previous_experiments (parent=True)

In [20]:
tst.run (test_register_manager, tag='dummy')